In [4]:
import os

In [5]:
%pwd

'/Users/ujjwaltiwari/projects/Kidney-Disease-Classifier/research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'/Users/ujjwaltiwari/projects/Kidney-Disease-Classifier'

In [8]:
# entity is the return type of data ingestion function

In [40]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_files: Path
    unzip_dir: Path


In [41]:
from DiseaseClassifier.constants import *
from DiseaseClassifier.utils.common import read_yaml, create_directories

In [42]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL = config.source_URL,
            local_data_files = config.local_data_files,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [43]:
import os
import zipfile
import gdown
from DiseaseClassifier import logger
from DiseaseClassifier.utils.common import get_size

In [44]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''
        try :
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_files
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Dowmloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into thr data directoy
        Function return None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_files, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [46]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-06 17:14:05,203: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-06 17:14:05,205: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-06 17:14:05,205: INFO: common: Created directory at : artifacts]
[2024-04-06 17:14:05,206: INFO: common: Created directory at : artifacts/data_ingestion]
[2024-04-06 17:14:05,206: INFO: 1026099227: Downloading data from https://drive.google.com/file/d/1RjY6ZECBmV9cLQHTLOh8VpOHd2cQkEeU/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1RjY6ZECBmV9cLQHTLOh8VpOHd2cQkEeU
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1RjY6ZECBmV9cLQHTLOh8VpOHd2cQkEeU&confirm=t&uuid=51432170-bbea-4134-aca9-0acea4eb46f9
To: /Users/ujjwaltiwari/projects/Kidney-Disease-Classifier/artifacts/data_ingestion/data.zip
100%|██████████| 1.63G/1.63G [05:08<00:00, 5.27MB/s]

[2024-04-06 17:19:18,461: INFO: 1026099227: Dowmloaded data from https://drive.google.com/file/d/1RjY6ZECBmV9cLQHTLOh8VpOHd2cQkEeU/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
